# Automating the Boring Stuff with Large Language Models(대규모 언어 모델로 작업 자동화하기)

예상 소요 시간: 60 분


이 프로젝트에서는 대규모 언어 모델을 활용하여 콜 센터 내의 업무를 개선할 것입니다. 콜 센터 내의 단순 반복적인 업무를 자동화 함으로써 좀더 중요한 업무를 위한 시간을 확보할 수 있습니다.

# watsonx API Authentication

watsonx API 호출을 위한 정보를 입력합니다.(강사가 알려주는 값으로 변경하시기 바랍니다.)

`/home/project/` 위치에 `.env` 파일을 만들고 아래 내용을 복사해서 붙여넣기 하세요.    
```
# /home/project/.env
PROJECT_ID="Your-Project-ID"
IBMCLOUD_API_KEY="Your-API-KEY"
API_ENDPOINT="https://eu-de.ml.cloud.ibm.com"
```

In [25]:
# 강사가 알려주는 값으로 아래 내용을 변경하시기 바랍니다.
PROJECT_ID="Your-Project-ID"
IBMCLOUD_API_KEY="Your-API-KEY"
API_ENDPOINT="https://us-south.ml.cloud.ibm.com"

# python 환경 설정

Command Palette 실행    
![](img/2024-05-23-14-10-56.png)

입력창에 python: Create 입력 후 아래 목록에서 선택    
![](img/2024-05-23-14-12-07.png)

venv 선택    
![](img/2024-05-23-14-10-17.png)

Python 3.10.x 선택    
![](img/2024-07-04-15-22-46.png)

vscode 오른쪽 상단 창에 선택된 Kernal 확인    
![](img/2024-07-04-15-23-32.png)

# 모델 선택

이 프로젝트에서는 벤치마크에서 우수한 성능과 상업적으로 허용되는 라이선스로 인해 현재 가장 인기 있는 오픈 소스 모델 중 하나이며 watsonx.ai에서 사용할 수 있는 기본 모델 중 하나인 `llama-2-70b-chat` or `llama-3-70b-instruct` 모델을 사용할 것입니다. 이 모델은 크기가 크고 실행에 상당한 리소스가 필요하지만, watsonx.ai와 `ibm-watson-machine-learning` 파이썬 라이브러리를 통해 이 모델과 쉽게 상호 작용할 수 있습니다.


## Llama 모델 고려사항
공식 Llama 깃허브 리포지토리는 프롬프트 생성에 대해 다음과 같이 작성할 것을 권장합니다.:

- **시스템 프롬프트**를 표시하려면 `<<SYS>>` / `<</SYS>>` 태그를 사용합니다. 시스템 프롬프트는 개발자가 LLM에 동작 방법을 알려주기 위해 작성하는 것입니다. 시스템 프롬프트에서 LLM이 수행하거나 수행하지 않기를 원하는 작업(예: "답변은 간결하고 논리적으로 작성하세요", "여러 단락의 독백으로 진행하지 마세요")을 지시할 수 있습니다.
- 사용자 생성 텍스트를 표시하려면 `[INST]` / `[/INST]` 태그를 사용합니다.
- AI가 생성한 응답에는 태그를 사용하지 마세요. 여기서 이해해야 할 다른 사항은 없으며, 대화 기록에서 AI가 생성한 응답에 대해 특별한 작업을 하지 마세요.

따라서 챗봇을 구축하는 경우 몇 개의 메시지를 보낸 후 watsonx.ai가 호스팅하는 llama 에게 보내는 프롬프트는 다음과 같이 표시될 수 있습니다:


> <<SYS>> You are an Al assistant.
> Provide useful and accurate answers to the user’s questions <</SYS>>
> 
> [INST] What is the capital of France? [/INST]
> 
> The capital of France is Paris
> 
> [INST] Is that also the most populous city on the continent? [/INST]
> 
> No, Istanbul in Turkey holds that title.
> It spans two continents, Europe and Asia,
> and its European part is more populous than Paris.



llama 저장소에서는 이러한 특수 토큰을 주입하려는 시도에 대한 사용자 입력을 모니터링할 것을 권장합니다. 사용자가 응답에 이러한 `<<SYS>>` 태그를 포함할 수 있는 경우, 후속 프롬프트에 대한 LLM의 동작을 변경할 수 있습니다.



# watsonx.ai Foundation Model 사용을 위한 ibm_watson_machine_learning Python Library 설치

IBM Watson Python API는 파이썬에서 IBM의 watson 서비스와 쉽게 상호 작용할 수 있는 강력한 도구입니다. IBM watson은 자연어 처리 기능을 포함하지만 이에 국한되지 않는 AI 도구 모음입니다. Python API는 Python 코드와 이러한 watson 서비스 사이의 가교 역할을 하여 사용자가 자체 애플리케이션에서 사용할 수 있도록 합니다.

## Installing requirements
다음 코드 스니펫을 실행하여 이 실습에 필요한 요구 사항을 설치하세요:



In [ ]:
!pip install ibm-watson-machine-learning
!pip install python-dotenv


- `ibm-watson-machine-learning` 라이브러리는 watsonx.ai와 상호 작용하기 위한 것입니다.
- `python-dotenv` 라이브러리는 `.env` 파일에서 watsonx.ai 자격 증명을 편리하게 읽기 위한 라이브러리입니다.

## API를 활용한 텍스트 생성

아래 예제 파이썬 코드는 형식이 잘 지정된 프롬프트와 함께 watsonx.ai 호스팅 LLama 모델을 사용하는 방법을 보여주는 예시입니다.


In [ ]:
# /home/project/example.py
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from dotenv import load_dotenv
import os

# loads our api credentials from .env into our environment
load_dotenv()

# Llama system instruction tokens
BEGIN_SYSTEM_INSTRUCTIONS = "<<SYS>>"
END_SYSTEM_INSTRUCTIONS = "<</SYS>>"

# Llama user prompt/instructions
BEGIN_USER_PROMPT = "[INST]"
END_USER_PROMPT = "[/INST]"

def prompt_contains_llama_tokens(prompt):
    """returns true if the prompt contains llama tokens (which may be malicious!)"""
    llama_tokens = [BEGIN_USER_PROMPT, END_USER_PROMPT, BEGIN_SYSTEM_INSTRUCTIONS, END_SYSTEM_INSTRUCTIONS]
    return any(token in prompt for token in llama_tokens)

def format_prompt(system_instructions, user_prompt, response_prefix=None):
    return f"{BEGIN_SYSTEM_INSTRUCTIONS}\n{system_instructions.strip()}\n{END_SYSTEM_INSTRUCTIONS}\n\n{BEGIN_USER_PROMPT}{user_prompt.strip()}{END_USER_PROMPT}\n{response_prefix if response_prefix else ''}"


generate_params = {
    GenParams.MAX_NEW_TOKENS: 900 # 900 is the max number of output tokens for LLAMA_2_70B_CHAT for the moment
}
model_id = "meta-llama/llama-3-70b-instruct"

# model = Model(
#     model_id=model_id,
#     params=generate_params,
#     credentials={
#         "apikey": os.environ["IBMCLOUD_API_KEY"],
#         "url": "https://us-south.ml.cloud.ibm.com"
#     },
#     project_id=os.environ["PROJECT_ID"]
#     )

model = Model(
    model_id=model_id,
    params=generate_params,
    credentials={
        "apikey": f"{IBMCLOUD_API_KEY}",
        "url": f"{API_ENDPOINT}"
    },
    project_id=f"{PROJECT_ID}"
    )

##############################################################
# Please have fun experimenting with the three variables below
##############################################################
# system_prompt controls the behaviour of the LLM
# system_prompt = "You are an AI assistant. Answer all prompts as succinctly as possible, without unnecessary commentary."
system_prompt = "귀하는 인공지능 비서입니다. 모든 프롬프트에 불필요한 설명 없이 가능한 한 간결하게 대답하세요. 모든 답변은 한국어로 합니다."

# user_prompt is the
# user_prompt = "What year was Las Vegas founded?"
user_prompt = "라스베가스는 언제 생겼나요?"

# response prefix lets you write the beginning of the LLM's response
# to force the LLM answer to take a certain form
# response_prefix = "Las Vegas was founded in"
response_prefix = "라스베가스는 "


if prompt_contains_llama_tokens(user_prompt):
    print("potentially malicious input detected")
else:
    formatted_prompt = format_prompt(system_prompt, user_prompt, response_prefix)
    completed_response = model.generate(prompt=formatted_prompt)['results'][0]['generated_text'].strip()
    composed_response = f"{response_prefix} {completed_response}"
    print(composed_response)


위 코드를 실행하면 "라스베가스는 몇 년에 설립되었나요?"라는 사용자 프롬프트에 대한 답변을 얻을수 있습니다. (또는 `user_prompt` / `response_prompt` 변수에 원하는 프롬프트 입력 후) 코드를 실행해 보세요:

아래의 문장이 결과입니다(실제로 라스베이거스가 설립된 해입니다):

> Las Vegas was founded in 1905.    
> 라스베가스는 1905년에 설립되었습니다.

실제 다음 과정에 들어가기 전에 모델의 기능과 동작을 파악하고 싶다면 이 시점에서 여러가지 프롬프트를 가지고 테스트해 보세요.

# 콜센터 유즈케이스
콜센터 유즈케이스를 살펴보겠습니다. 두 가지 관련 직무를 살펴보고 watsonx.ai의 도움으로 이들의 업무를 더 쉽게 만들 수 있는 방법을 살펴보겠습니다.

## 역할

### Call Center Agent(콜센터 상담원)
콜센터 상담원은 고객의 전화를 받고 문제를 해결하는 일을 담당합니다. 몇 가지 담당하는 업무는 다음과 같습니다:

- 서비스 문제로 인한 것이 아닌 고객의 문제에 대해 상담하기(예: 기술에 익숙하지 않은 고객은 WiFi 연결 방법을 모를 수 있음).
- 서비스 문제에 대한 서비스 콜 예약하기
- 나중에 참조할 수 있도록 고객의 전화 사유를 메모합니다(특히 고객이 다시 전화해야 하는 경우).

### Call Center Manager(콜센터 관리자)
- 콜센터 통화 모니터링 및 상담원 성과 평가를 담당합니다.
- 상담원의 성과 향상을 위한 코칭을 담당합니다.
- 실제로는 이보다 훨씬 더 많지만 위의 두 가지 사항을 개선하려고 노력할 것입니다.

## 샘플 데이터
콜센터에는 다양한 유형의 데이터가 있습니다. 이 실습에서는 예시를 위해 생성된 콜센터 데이터를 사용합니다.

아래의 리포지토리를 복제하여 액세스하세요:

In [ ]:
git clone https://github.com/ibm-developer-skills-network/mvqmg-call-center-llm-samples.git call-center-llm-samples

### Call Transcripts(통화 녹취록)

`call-center-llm-samples/data/transcripts` 디렉터리에 고객과 상담원 간의 짧은 통화 기록이 몇 개 있습니다.

1.txt 파일을 IDE로 열어서 내용을 살펴봅니다.

### Knowledgebase(지식베이스)

`call-center-llm-samples/data/knowledgebase.md`는 가상의 제품/기능에 대해 자주 묻는 20가지 질문으로 구성된 지식베이스 입니다.

knowledgebase.md 파일을 IDE로 열어서 내용을 살펴봅니다.

다음에는 이러한 데이터와 함께 LLM 및 watsonx.ai를 사용하여 콜센터 상담원과 관리자가 완료해야 하는 몇 가지 지루한 작업을 자동화하는 방법을 살펴보겠습니다.

# 시나리오 1: 상담 노트


상담 노트는 상담원이 사례/통화에 대해 작성하는 정보입니다. 이러한 메모는 고객이 다시 전화를 걸어 다른 상담원에게 연결되는 경우(또는 같은 상담원이 이전 내용을 잊어버린 경우) 다른 상담원에게 참고 자료로 사용될 수 있습니다. 또한 경우에 따라 경영진이 검토할 수도 있습니다. 간단해 보이지만 콜센터의 근무 환경을 고려하면 생각보다 어려울 수 있습니다.

콜센터 상담원은 몇 가지 업무 상태에 있을 수 있으며, 각 상태에서 상담원이 보내는 시간이 모니터링됩니다.


- 대기 중: 전화를 받을 준비가 되었지만 대기 중인 고객이 없는 상태
- 통화 중: 고객과 적극적으로 대화하며 문제 해결을 돕고 있음
- 문제 해결 중(고객 통화 중): 고객이 통화 대기 중인 동안 문제 해결 중
- 통화 종료 후: 통화가 끝난 후 상담원이 이 상태로 전환 되며, 상담원이 준비가 되면 대기열에 다시 참여
- 휴식 중


상담원이 마음대로 상담 노트를 적을 수 있는 "상담 노트 작성 중"이라는 특별한 상태는 없습니다. 즉 상담원은 다음 중 하나를 수행해야 합니다:


- 고객과 통화하는 동시에 상담 노트를 작성하면 상담원의 주의가 산만해지고 고객이 불만을 가질 수 있습니다.
- 고객이 대기 중일 때 상담 노트를 작성하여 통화 시간이 길어지고 고객에게 불쾌감을 줍니다.
- 통화 사이에 상담 노트를 작성하면 통화 간 시간이 늘어나고(상담원에게 부정적인 영향을 미침) 처리되는 통화 처리량이 감소합니다(고객의 대기 시간이 길어짐).


상담원 업무의 일부인 상담 노트를 자동화하거나 최소한 상담원이 참고할 수 있는 좋은 요약을 제공할 수 있는지 살펴봅시다.



이 시나리오의 목표는 `call-center-llm-samples/data/transcripts`에 있는 각 녹취록을 한 문장으로 요약하는 것입니다(사례 메모 역할을 할 수 있음).
한글파일은 `call-center-llm-samples/data/transcripts-kr`에 있음


이 시나리오를 처리하는 코드를 아래를 참고하시기 바랍니다.

원한다면 이 예제 코드를 사용하여 바로 시작할 수 있습니다. `TODO` 부분에 프롬프트만 채우면 됩니다:

In [ ]:
# /home/project/summarize.py
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from dotenv import load_dotenv
import os

load_dotenv()

BEGIN_USER_INSTRUCTIONS = "[INST]"
END_USER_INSTRUCTIONS = "[/INST]"
BEGIN_SYSTEM_INSTRUCTIONS = "<<SYS>>"
END_SYSTEM_INSTRUCTIONS = "<</SYS>>"

def prompt_contains_llama_tokens(prompt):
    llama_tokens = [BEGIN_USER_INSTRUCTIONS, END_USER_INSTRUCTIONS, BEGIN_SYSTEM_INSTRUCTIONS, END_SYSTEM_INSTRUCTIONS]
    return any(token in prompt for token in llama_tokens)

def format_prompt(system_instructions, user_prompt, response_prefix=None):
    return f"{BEGIN_SYSTEM_INSTRUCTIONS}\n{system_instructions.strip()}\n{END_SYSTEM_INSTRUCTIONS}\n\n{BEGIN_USER_INSTRUCTIONS}{user_prompt.strip()}{END_USER_INSTRUCTIONS}\n{response_prefix if response_prefix else ''}"


generate_params = {
    GenParams.MAX_NEW_TOKENS: 900 # 900 is the max number of output tokens for LLAMA_2_70B_CHAT for the moment
}
# llama-3-70b-instruct, LLAMA_2_70B_CHAT, LLAMA_3_70B_INSTRUCT
model_id = "meta-llama/llama-3-70b-instruct"


model = Model(
    model_id=model_id,
    params=generate_params,
    credentials={
        "apikey": f"{IBMCLOUD_API_KEY}",
        "url": f"{API_ENDPOINT}"
    },
    project_id=f"{PROJECT_ID}"
    )

# system_prompt controls the behaviour of the LLM
system_prompt = # TODO: set a system prompt

# response prefix lets you write the beginning of
# the response on the AI's behalf to force the answer to take a certain form
response_prefix = "" # TODO: optionally set a response prefix


PATH_TO_TRANSCRIPTS = "call-center-llm-samples/data/transcripts"
for filename in sorted(os.listdir(PATH_TO_TRANSCRIPTS)):
    with open(f"{PATH_TO_TRANSCRIPTS}/{filename}", "r", encoding="utf-8") as transcript:
        user_prompt = transcript.read()

        if prompt_contains_llama_tokens(user_prompt): # seems unlikely, but just to be safe...
            print("potentially malicious input detected")
        else:
            formatted_prompt = format_prompt(system_prompt, user_prompt, response_prefix)
            completed_response = model.generate(prompt=formatted_prompt)['results'][0]['generated_text'].strip()
            composed_response = f"{completed_response}"
            print(f"summary of transcript {filename}:")
            print(composed_response)


`TODO` 섹션을 작성한 후 실행하여 코드를 테스트할 수 있습니다:


다음 내용은 위의 시나리오에 대한 솔루션중 한가지 입니다.

# 시니라오 1: 상담 노트 - 솔루션


아래는 가능한 솔루션 중 하나일 뿐입니다. 자체 프롬프트의 출력과 비교하여 어느 쪽이 더 나은지 확인해 보시기 바랍니다.

In [ ]:
# /home/project/summarize.py
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from dotenv import load_dotenv
import os

load_dotenv()

BEGIN_USER_INSTRUCTIONS = "[INST]"
END_USER_INSTRUCTIONS = "[/INST]"
BEGIN_SYSTEM_INSTRUCTIONS = "<<SYS>>"
END_SYSTEM_INSTRUCTIONS = "<</SYS>>"

def prompt_contains_llama_tokens(prompt):
    llama_tokens = [BEGIN_USER_INSTRUCTIONS, END_USER_INSTRUCTIONS, BEGIN_SYSTEM_INSTRUCTIONS, END_SYSTEM_INSTRUCTIONS]
    return any(token in prompt for token in llama_tokens)

def format_prompt(system_instructions, user_prompt, response_prefix=None):
    return f"{BEGIN_SYSTEM_INSTRUCTIONS}\n{system_instructions.strip()}\n{END_SYSTEM_INSTRUCTIONS}\n\n{BEGIN_USER_INSTRUCTIONS}{user_prompt.strip()}{END_USER_INSTRUCTIONS}\n{response_prefix if response_prefix else ''}"


generate_params = {
    GenParams.MAX_NEW_TOKENS: 900 # 900 is the max number of output tokens for LLAMA_2_70B_CHAT for the moment
}
# llama-3-70b-instruct, LLAMA_2_70B_CHAT, LLAMA_3_70B_INSTRUCT
model_id = "meta-llama/llama-3-70b-instruct"

model = Model(
    model_id=model_id,
    params=generate_params,
    credentials={
        "apikey": f"{IBMCLOUD_API_KEY}",
        "url": f"{API_ENDPOINT}"
    },
    project_id=f"{PROJECT_ID}"
    )


# system_prompt controls the behaviour of the LLM
# system_prompt = "You are an AI assistant at an ISP call center specializing in transcript summarization. You are provided a transcript and will return a single sentence summary of the call. You must respond with only the summary, no greeting or commentary."
system_prompt = "귀하는 대화 내용 요약 전문 ISP 콜센터의 AI 어시스턴트입니다. 모든 답변은 한국어로 합니다. 녹취록이 제공되며 통화에 대해 한 문장 요약본을 반환합니다. 인사말이나 해설 없이 요약만 응답해야 합니다."
# response prefix lets you write the beginning of 
# the response on the AI's behalf to force the answer to take a certain form
# response_prefix = "One sentence summary: "
response_prefix = "한 문장 요약: "

PATH_TO_TRANSCRIPTS = "call-center-llm-samples/data/transcripts-kr"

for filename in sorted(os.listdir(PATH_TO_TRANSCRIPTS)):
    with open(f"{PATH_TO_TRANSCRIPTS}/{filename}", "r", encoding="utf-8") as transcript:
        user_prompt = transcript.read()

        if prompt_contains_llama_tokens(user_prompt): # seems unlikely, but just to be safe...
            print("potentially malicious input detected")
        else:
            formatted_prompt = format_prompt(system_prompt, user_prompt, response_prefix)
            completed_response = model.generate(prompt=formatted_prompt)['results'][0]['generated_text'].strip()
            composed_response = f"{completed_response}"
            print(f"PROMPT BEGIN:")
            print(f"{formatted_prompt}")
            print(f"PROMPT END")
            print(f"summary of transcript {filename}:")
            print(composed_response)

## 아래는 결과 예시 입니다.
```
summary of transcript 1.txt:
ISP 콜센터의 고객 지원 에이전트가 고객의 인터넷 속도 문제를 해결했습니다.
summary of transcript 2.txt:
John Doe는 ISP 지원 센터에 연락하여 지역의 회선 문제로 인한 느린 인터넷 연결을 신고했습니다.
summary of transcript 3.txt:
ISP 콜센터의 고객은 인터넷 연결 문제를 신고했으며 에이전트는 문제 해결을 위해 라우터 재시작, 케이블 점검, 공장 초기화 등의 단계를 안내했습니다.
summary of transcript 4.txt:
John Doe는 인터넷 연결 문제를 신고하여 ISP 콜센터에 연락했으며, 에이전트는 문제를 해결하고 고객에게 서비스 만족도 조사에 대한 안내를 제공했습니다.


summary of transcript 1.txt:
The customer's internet speed was improved by the agent's adjustments, and the customer was notified about receiving a survey regarding the call.
summary of transcript 2.txt:
A customer named John Doe called the ISP support center to report a slow internet connection, which the agent traced to a line issue in his area and promised to have fixed soon.
summary of transcript 3.txt:
The customer called the support line because of an internet connection issue that was dropping constantly, and after trying a few troubleshooting steps, the issue was resolved by resetting the router to factory settings.
summary of transcript 4.txt:
The customer, John Doe, called the support center because of a slow and disconnecting internet connection, which the agent resolved by resetting the connection, and the customer was satisfied with the resolution.
```

# 시니라오 2: 지식베이스 지원

이 시나리오의 목표는 LLM이 지식베이스에 포함된 정보를 사용하여 질문에 답변하도록 하는 것입니다. 이 지식베이스는 LLM의 학습 데이터에 없는 가상의 제품 및 기능으로 채워져 있습니다.

원한다면 이 예제 코드를 사용하여 바로 시작할 수 있습니다. `TODO` 부분에 프롬프트만 채우면 됩니다:

In [ ]:
# /home/project/knowledgebase.py
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from dotenv import load_dotenv
import os

load_dotenv()

BEGIN_USER_INSTRUCTIONS = "[INST]"
END_USER_INSTRUCTIONS = "[/INST]"
BEGIN_SYSTEM_INSTRUCTIONS = "<<SYS>>"
END_SYSTEM_INSTRUCTIONS = "<</SYS>>"

def prompt_contains_llama_tokens(prompt):
    llama_tokens = [BEGIN_USER_INSTRUCTIONS, END_USER_INSTRUCTIONS, BEGIN_SYSTEM_INSTRUCTIONS, END_SYSTEM_INSTRUCTIONS]
    return any(token in prompt for token in llama_tokens)

def format_prompt(system_instructions, user_instructions, response_prefix=None):
    return f"{BEGIN_SYSTEM_INSTRUCTIONS}\n{system_instructions.strip()}\n{END_SYSTEM_INSTRUCTIONS}\n\n{BEGIN_USER_INSTRUCTIONS}{user_instructions.strip()}{END_USER_INSTRUCTIONS}\n{response_prefix if response_prefix else ''}"


generate_params = {
    GenParams.MAX_NEW_TOKENS: 900 # 900 is the max number of output tokens for LLAMA_2_70B_CHAT for the moment
}

model = Model(
    model_id=model_id,
    params=generate_params,
    credentials={
        "apikey": f"{IBMCLOUD_API_KEY}",
        "url": f"{API_ENDPOINT}"
    },
    project_id=f"{PROJECT_ID}"
    )

# 영문, 한글 변경시 수정
with open("call-center-llm-samples/data/knowledgebase.md", "r", encoding="utf-8") as kb:
    kb_text = kb.read()

SYSTEM_PROMPT = f"""
# TODO: add your system prompt here
"""


USER_PROMPT = "How can I activate waveguard?" # Or ask a different question that can be answered by referencing the knowledgebase

if prompt_contains_llama_tokens(USER_PROMPT):
    print("potentially malicious input detected")
else:
    formatted_prompt = # TODO: generate/format your prompt
    generated_response = model.generate(prompt=formatted_prompt)
    print(generated_response['results'][0]['generated_text'].strip())

`TODO` 섹션을 작성한 후 실행하여 코드를 테스트할 수 있습니다:


다음 내용은 위의 시나리오에 대한 솔루션중 한가지 입니다. 여러분이 작성한 위의 결과와 비교해 보시기 바랍니다.

# 시나리오 2: 지식베이스 지원 - 솔루션

In [ ]:
# /home/project/knowledgebase.py
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from dotenv import load_dotenv
import os

load_dotenv()

BEGIN_USER_INSTRUCTIONS = "[INST]"
END_USER_INSTRUCTIONS = "[/INST]"
BEGIN_SYSTEM_INSTRUCTIONS = "<<SYS>>"
END_SYSTEM_INSTRUCTIONS = "<</SYS>>"

def prompt_contains_llama_tokens(prompt):
    llama_tokens = [BEGIN_USER_INSTRUCTIONS, END_USER_INSTRUCTIONS, BEGIN_SYSTEM_INSTRUCTIONS, END_SYSTEM_INSTRUCTIONS]
    return any(token in prompt for token in llama_tokens)

def format_prompt(system_instructions, user_instructions, response_prefix=None):
    return f"{BEGIN_SYSTEM_INSTRUCTIONS}\n{system_instructions.strip()}\n{END_SYSTEM_INSTRUCTIONS}\n\n{BEGIN_USER_INSTRUCTIONS}{user_instructions.strip()}{END_USER_INSTRUCTIONS}\n{response_prefix if response_prefix else ''}"


generate_params = {
    GenParams.MAX_NEW_TOKENS: 1024 # 900 is the max number of output tokens for LLAMA_2_70B_CHAT for the moment
}

model_id = "meta-llama/llama-3-70b-instruct"

model = Model(
    model_id=model_id,
    params=generate_params,
    credentials={
        "apikey": f"{IBMCLOUD_API_KEY}",
        "url": f"{API_ENDPOINT}"
    },
    project_id=f"{PROJECT_ID}"
    )

# 영문 knowledgebase를 사용할 경우 아래 파일명을 knowledgebase.md로 변경 필요
with open("call-center-llm-samples/data/knowledgebase-kr.md", "r", encoding="utf-8") as kb:
    kb_text = kb.read()

# SYSTEM_PROMPT = f"""
# Answer questions as a call center AI assistant. 
# Try to answer as succinctly as possible. 
# Do not provide generic advice. You only use the below knowledgebase to answer questions. 
# Only answer with information in the knowledgebase. 
# If a question is not answered in the knowledgebase, respond that you do not know.

# {kb_text}
# """

SYSTEM_PROMPT = f"""
콜센터 AI 도우미로서 질문에 답변하세요. 
가능한 한 간결하게 답변하세요. 
일반적인 조언은 제공하지 마세요. 아래 지식창고에 있는 정보만 사용하여 질문에 답변하세요. 
지식창고에 있는 정보로만 답변하세요. 
지식창고에 답변이 없는 질문은 모른다고 답변하세요.

{kb_text}
"""

# USER_PROMPT = "How can I activate waveguard?"
USER_PROMPT = "웨이브가드를 활성화하려면 어떻게 해야 하나요?"

if prompt_contains_llama_tokens(USER_PROMPT):
    print("potentially malicious input detected")
else:
    formatted_prompt = format_prompt(SYSTEM_PROMPT, USER_PROMPT)
    generated_response = model.generate(prompt=formatted_prompt)
    print(f"PROMPT BEGIN:")
    print(f"{formatted_prompt}")
    print(f"PROMPT END")
    print(generated_response['results'][0]['generated_text'].strip())

## 답변 예시
WaveGuard를 활성화하면 다른 무선 네트워크 및 장치로부터의 간섭을 최소화할 수 있습니다. 192.168.55.1에서 라우터 설정에 액세스하고 "admin"과 "cr55guard"를 자격 증명으로 사용하십시오. "무선" 탭 아래에서 "WaveGuard" 섹션을 찾을 수 있습니다. 이를 활성화하고 보호 반경을 "중간"으로 설정하여 보호와 무선 범위 간의 균형을 맞추십시오. 설정을 적용하고 저장한 후 라우터를 재시작하여 변경 사항을 적용하십시오.

You can activate WaveGuard interference protection on the CR-550 router by accessing the router settings at `192.168.55.1` using "admin" and "cr55guard" as credentials. Under the "Wireless" tab, you'll find the "WaveGuard" section. Enable it and set the protective radius to "Medium" for a balanced approach between protection and wireless range. Apply and save the settings, then restart the router for the changes to take effect.


이 솔루션을 사용해 보려면 위의 코드를 실행하면 됩니다:

계속 진행하기 전에 잠시 동안 이 프롬프트를 자유롭게 실험해 보세요.

# 시나리오 3: 상담 평가


콜센터 관리자는 상담원이 모든 일을 올바르게 수행하고 있는지 확인하기 위해 상담원의 통화를 지속적으로 검토해야 합니다.

관리자가 준수 여부를 확인하는 몇 가지 일반적인 사례는 다음과 같습니다:


- 상담원이 문제 해결을 시작하기 전에 고객의 신원을 제대로 확인했는가?
- 상담원이 '안심 진술서'를 사용했는가? 여기서 안심 진술이란 상담원이 고객을 안심시키고 기대치를 설정하기 위해 사용하는 커뮤니케이션 도구입니다. 예를 들어 "오늘 이 문제를 해결해 드리겠습니다."와 같은 문구입니다.
- 상담원이 고객의 통화 사유가 해결되었음을 확인했나요?
- 상담원이 고객에게 고객 서비스 품질에 대한 설문조사를 받게 될 것이라고 안내했나요? 상담원이 이를 언급하지 않으면 고객은 특정 상담원의 고객 서비스가 아닌 회사 전체를 평가하는 경우가 많습니다.


콜센터 관리자는 자신이 담당하는 상담원이 수십 명에 달할 수 있으며, 모든 통화를 일일이 검토할 시간이 없습니다. 실제로는 직관/휴리스틱(예: "이 통화는 왜 이렇게 길었나? 짧았나?")을 사용하여 일부 통화를 선택해야 합니다. ("통화가 너무 짧은데?" 또는 "이 상담원은 설문조사에서 낮은 점수를 받았으니 몇 개의 통화를 검토해 봐야겠어").

이 작업을 자동화해 보겠습니다:

- 상담원이 모범 사례를 따르지 않을 경우 자동화된 피드백/알람을 받을 수 있습니다.
- 관리자는 전체 녹취록을 듣거나 읽지 않고도 통화에 대한 더 나은 정보를 얻을 수 있습니다.


이 시나리오의 목표는 `call-center-llm-samples/data/transcripts`의 각 녹취록이 필수 항목을 얼마나 잘 준수하는지 확인하는 파이썬 어플리케이션을 만드는 것입니다.


자동화 시 콜센터 내의 기존 API와 통합할 수 있기 때문에 구조화된 출력을 얻는 것은 중요합니다. 즉, 다른 시스템과 연계를 위해서는 다음과 같은 작업을 수행할 수 있어야 합니다:
```
# (not a real API)
curl -X PATCH "https://api.callcenter.com/calls/12345" \
     -H "Content-Type: application/json" \
     -H "Authorization: Bearer your_access_token" \
     -d '{
           "identityConfirmed": false,
           "reassuranceStatement": false,
           "reasonForCallResolved": true,
           "advisedOfSurvey": true
         }'
```


그러나, LLM에서 구조화된 출력을 안정적으로 얻는 것은 어려울 수 있습니다. 유효한 JSON을 생성할 수 있는 경우에도 생성되는 JSON에 대해 설명하거나 `OK. Here JSON: ...`와 같이 응답하여 구문 분석을 복잡하게 만듭니다.

결과를 JSON으로 강제 변환하기 위해 고려할 수 있는 몇 가지 옵션이 있습니다:

1. Stop sequence사용:
   ```python
    # introduce a new stop sequence by setting "stop_sequences" in the generate_params variable
    generate_params = {
      GenParams.MAX_NEW_TOKENS: 900, # 900 is the max number of output tokens for LLAMA_2_70B_CHAT for the moment
      "stop_sequences": [] # <--- The LLM will stop outputting text when it produces a string that's in this array
    }
   ```

2. Few-shot 프롬프트: 프롬프트에 원하는 출력의 몇 가지 예를 제시하고 모델이 원하는 출력이 무엇인지 파악하는지 확인합니다.

3. The Non-LLM 방법: normal/non-LLM 파이썬 코드를 사용하여 LLM 결과에서 유효한 JSON을 추출해 보세요.

원한다면 이 예제 코드를 사용하여 바로 시작할 수 있습니다. `TODO` 부분에 프롬프트만 채우면 됩니다:

In [ ]:
# /home/project/evaluate.py
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from dotenv import load_dotenv
import os
import json

load_dotenv()

BEGIN_USER_INSTRUCTIONS = "[INST]"
END_USER_INSTRUCTIONS = "[/INST]"
BEGIN_SYSTEM_INSTRUCTIONS = "<<SYS>>"
END_SYSTEM_INSTRUCTIONS = "<</SYS>>"

def prompt_contains_llama_tokens(prompt):
    llama_tokens = [BEGIN_USER_INSTRUCTIONS, END_USER_INSTRUCTIONS, BEGIN_SYSTEM_INSTRUCTIONS, END_SYSTEM_INSTRUCTIONS]
    return any(token in prompt for token in llama_tokens)

def format_prompt(system_instructions, user_instructions, response_prefix=None):
    return f"{BEGIN_SYSTEM_INSTRUCTIONS}\n{system_instructions.strip()}\n{END_SYSTEM_INSTRUCTIONS}\n\n{BEGIN_USER_INSTRUCTIONS}{user_instructions.strip()}{END_USER_INSTRUCTIONS}\n{response_prefix if response_prefix else ''}"


generate_params = {
    GenParams.MAX_NEW_TOKENS: 900 # 900 is the max number of output tokens for LLAMA_2_70B_CHAT for the moment
}

model = Model(
    model_id=model_id,
    params=generate_params,
    credentials={
        "apikey": f"{IBMCLOUD_API_KEY}",
        "url": f"{API_ENDPOINT}"
    },
    project_id=f"{PROJECT_ID}"
    )



# Feel free to experiment with these
SYSTEM_PROMPT = """
    # TODO: add your system prompt here
    """
ANSWER_PREFIX = "# TODO: add your answer prefix here"

PATH_TO_TRANSCRIPTS = "call-center-llm-samples/data/transcripts"
for filename in sorted(os.listdir(PATH_TO_TRANSCRIPTS)):
    with open(f"{PATH_TO_TRANSCRIPTS}/{filename}", "r", encoding="utf-8") as transcript:
        transcript_text = transcript.read()
        if prompt_contains_llama_tokens(transcript): # seems unlikely, but just to be safe...
            print("potentially malicious input detected")
        else:
            formatted_prompt = # TODO: generate / format your prompt
            generated_response = model.generate(prompt=formatted_prompt)
            response_text = generated_response['results'][0]['generated_text'].strip()

            print(f"{filename}:")
            try:
                parsed_evaluation = json.loads(response_text)
                assert "identityConfirmed" in parsed_evaluation
                assert "reassuranceStatement" in parsed_evaluation
                assert "reasonForCallResolved" in parsed_evaluation
                assert "advisedOfSurvey" in parsed_evaluation
            except json.decoder.JSONDecodeError:
                print("Your response was not proper JSON.")
                print("Your response:")
                print(response_text)
            except AssertionError as e:
                print("One of the expected keys was not found.")
                print("Double check that you don't have a typo in your prompt")
                print(e)

            print(generated_response['results'][0]['generated_text'].strip())

## 예상 결과는 아래와 같음:
> ```shell
1.txt:
{“identityConfirmed”: false, “reassuranceStatement”: false, “reasonForCallResolved”: true, “advisedOfSurvey”: true }
 
2.txt:
{“identityConfirmed”: true, “reassuranceStatement”: false, “reasonForCallResolved”: false, “advisedOfSurvey”: false }
 
3.txt:
{“identityConfirmed”: false, “reassuranceStatement”: false, “reasonForCallResolved”: false, “advisedOfSurvey”: false }
 
4.txt:
{“identityConfirmed”: true, “reassuranceStatement”: true, “reasonForCallResolved”: true, “advisedOfSurvey”: true }
```



`TODO` 섹션을 작성한 후 실행하여 코드를 테스트할 수 있습니다:


다음 내용은 정확도가 100%는 아니지만 구현 예시가 나와 있습니다. 여러분이 작성한 위의 결과와 비교해 보시기 바랍니다.

# 시나리오 3: 상담 평가 - 솔루션

이 솔루션에서는 `}` Stop sequence 와 `JSON response:` answer prefix의 조합을 사용하여 모델이 강제로 JSON을 생성하도록 합니다.


In [ ]:
# /home/project/evaluate.py
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from dotenv import load_dotenv
import os
import json

load_dotenv()

BEGIN_USER_INSTRUCTIONS = "[INST]"
END_USER_INSTRUCTIONS = "[/INST]"
BEGIN_SYSTEM_INSTRUCTIONS = "<<SYS>>"
END_SYSTEM_INSTRUCTIONS = "<</SYS>>"

def prompt_contains_llama_tokens(prompt):
    llama_tokens = [BEGIN_USER_INSTRUCTIONS, END_USER_INSTRUCTIONS, BEGIN_SYSTEM_INSTRUCTIONS, END_SYSTEM_INSTRUCTIONS]
    return any(token in prompt for token in llama_tokens)

def format_prompt(system_instructions, user_instructions, response_prefix=None):
    return f"{BEGIN_SYSTEM_INSTRUCTIONS}\n{system_instructions.strip()}\n{END_SYSTEM_INSTRUCTIONS}\n\n{BEGIN_USER_INSTRUCTIONS}{user_instructions.strip()}{END_USER_INSTRUCTIONS}\n{response_prefix if response_prefix else ''}"


generate_params = {
    GenParams.MAX_NEW_TOKENS: 900, # 900 is the max number of output tokens for LLAMA_2_70B_CHAT for the moment
    "stop_sequences": ["}"] # <--- NOTE: for this example we've added a new stop sequence
}

model_id = "meta-llama/llama-3-70b-instruct"
# model_id = "meta-llama/llama-2-70b-chat"

model = Model(
    model_id=model_id,
    params=generate_params,
    credentials={
        "apikey": f"{IBMCLOUD_API_KEY}",
        "url": f"{API_ENDPOINT}"
    },
    project_id=f"{PROJECT_ID}"
    )



# Feel free to experiment with these
# SYSTEM_PROMPT = """
#     You are an AI assistant at an ISP call center specializing in evaluating call center agent performance based on call transcripts. 
#     You are provided a transcript and will return a JSON response.
#     The structure of the response is: {"identityConfirmed": boolean, "reassuranceStatement": boolean, "reasonForCallResolved": boolean, "advisedOfSurvey": boolean}
#     "identityConfirmed": true if the agent was able to confirm the customer's identity by asking their full name and PIN
#     "reassuranceStatement": true if the agent provided a statement of reassurance after the customer explained the problem (e.g. I can help you resolve that today)"
#     "reasonForCallResolved": true if the agent asked if the reason for the call was resolved
#     "advisedOfSurvey": true if the agent advised the customer that they will receive a survey about today's call and explained that the survey is for the agent's performance and not the quality of internet service
#     """
SYSTEM_PROMPT = """
    귀하는 통화 기록을 기반으로 콜센터 상담원의 성과를 전문적으로 평가하는 ISP 콜센터의 AI 어시스턴트입니다. 
    대화 내용을 제공받으면 JSON 응답을 반환합니다.
    응답의 구조는 다음과 같습니다: {"identityConfirmed": boolean, "reassuranceStatement": boolean, "reasonForCallResolved": boolean, "advisedOfSurvey": boolean}
    "identityConfirmed": 상담원이 고객의 이름과 PIN을 물어 고객의 신원을 확인했다면 true입니다.
    "reassuranceStatement": 고객이 문제를 설명한 후 상담원이 안심 안내를 제공한 경우(예: 오늘 해결을 도와드릴 수 있습니다)에 true입니다.
    "reasonForCallResolved": 상담원이 통화 사유가 해결되었는지 물어본 경우 true입니다.
    "advisedOfSurvey": 상담원이 고객에게 오늘 통화에 대한 설문조사를 받게 될 것이라고 알리고 설문조사가 인터넷 서비스 품질이 아닌 상담원의 성과를 위한 것이라고 설명한 경우 true입니다.
    """
ANSWER_PREFIX = "JSON response: "

PATH_TO_TRANSCRIPTS = "call-center-llm-samples/data/transcripts-kr"
for filename in sorted(os.listdir(PATH_TO_TRANSCRIPTS)):
    with open(f"{PATH_TO_TRANSCRIPTS}/{filename}", "r", encoding="utf-8") as transcript:
        transcript_text = transcript.read()
        if prompt_contains_llama_tokens(transcript): # seems unlikely, but just to be safe...
            print("potentially malicious input detected")
        else:
            formatted_prompt = format_prompt(SYSTEM_PROMPT, transcript_text, ANSWER_PREFIX)
            generated_response = model.generate(prompt=formatted_prompt)
            response_text = generated_response['results'][0]['generated_text'].strip()
            
            print(f"PROMPT BEGIN:")
            print(f"{formatted_prompt}")
            print(f"PROMPT END")
            print(f"{filename}:")
            try:
                parsed_evaluation = json.loads(response_text)
                assert "identityConfirmed" in parsed_evaluation
                assert "reassuranceStatement" in parsed_evaluation
                assert "reasonForCallResolved" in parsed_evaluation
                assert "advisedOfSurvey" in parsed_evaluation
            except json.decoder.JSONDecodeError:
                print("Your response was not proper JSON.")
                print("Your response:")
                print(response_text)
            except AssertionError as e:
                print("One of the expected keys was not found.")
                print("Double check that you don't have a typo in your prompt")
                print(e)

            print(generated_response['results'][0]['generated_text'].strip())

## 결과 예시: 
```

eng llama-2
1.txt:
{"identityConfirmed": false, "reassuranceStatement": true, "reasonForCallResolved": true, "advisedOfSurvey": true}
2.txt:
{"identityConfirmed": true, "reassuranceStatement": true, "reasonForCallResolved": true, "advisedOfSurvey": false}
3.txt:
{"identityConfirmed": false, "reassuranceStatement": true, "reasonForCallResolved": true, "advisedOfSurvey": false}
4.txt:
{"identityConfirmed": true, "reassuranceStatement": true, "reasonForCallResolved": true, "advisedOfSurvey": true}

eng llama-3
1.txt:
{"identityConfirmed": false, "reassuranceStatement": false, "reasonForCallResolved": false, "advisedOfSurvey": true}
2.txt:
{"identityConfirmed": true, "reassuranceStatement": true, "reasonForCallResolved": false, "advisedOfSurvey": false}
3.txt:
{"identityConfirmed": false, "reassuranceStatement": false, "reasonForCallResolved": false, "advisedOfSurvey": false}
4.txt:
{"identityConfirmed": true, "reassuranceStatement": true, "reasonForCallResolved": false, "advisedOfSurvey": true}

kr llama-3
1-kr.txt:
{"identityConfirmed": true, "reassuranceStatement": false, "reasonForCallResolved": false, "advisedOfSurvey": true}
2-kr.txt:
{"identityConfirmed": true, "reassuranceStatement": true, "reasonForCallResolved": true, "advisedOfSurvey": false}
3-kr.txt:
{"identityConfirmed": false, "reassuranceStatement": true, "reasonForCallResolved": true, "advisedOfSurvey": false}
4-kr.txt:
{"identityConfirmed": true, "reassuranceStatement": true, "reasonForCallResolved": true, "advisedOfSurvey": true}
```

위의 코드를 실행하여 결과를 확인합니다:

프롬프트, 모델 등을 변경하여 다양한 결과를 확인해 보세요. 
 

# 마무리


축하합니다!     
과정의 끝까지 잘 오셨습니다. 이 시나리오는 대규모 언어 모델의 기능을 사용하여 콜센터 내에서 이루어지는 다소 지루한 활동을 자동화하는 방법을 보여줍니다.

이 가이드 프로젝트에서 다루지 못한 것들이 많이 있을수 있습니다. 이 실습 환경에서 샘플 데이터로 계속해서 다음과 같은 몇 가지 방법을 시도해 볼 수 있습니다:


- 상담원을 위한 LLM 기반 교육 시뮬레이터:
- 훨씬 더 큰 규모의 트랜스크립트 작업을 위한 전략(LLM을 사용하여 대화의 앞부분을 요약, 또는 내용을 쉽게 조회할 수 있도록 녹취록의 임베딩을 벡터 데이터베이스에 저장)

다른 좋은 아이디어가 있으면 계속해서 테스트 해보시기 바랍니다!

감사합니다!
